In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist

import numpy as np

In [2]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [3]:
x_train = x_train.reshape(-1, 784) / 255.0  
x_test = x_test.reshape(-1, 784) / 255.0

In [4]:

y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)


In [5]:
len(y_train[0])

10

In [6]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(784,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [7]:
model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])
model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))


Epoch 1/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0259 - accuracy: 0.8188 - val_loss: 0.0220 - val_accuracy: 0.8495
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0199 - accuracy: 0.8626 - val_loss: 0.0205 - val_accuracy: 0.8565
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0182 - accuracy: 0.8746 - val_loss: 0.0192 - val_accuracy: 0.8660
Epoch 4/10
1875/1875 [==============================] - 7s 3ms/step - loss: 0.0171 - accuracy: 0.8821 - val_loss: 0.0183 - val_accuracy: 0.8741
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0163 - accuracy: 0.8881 - val_loss: 0.0186 - val_accuracy: 0.8706
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0155 - accuracy: 0.8940 - val_loss: 0.0177 - val_accuracy: 0.8782
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0151 - accuracy: 0.8968 - val_loss: 0.0190 - val_accuracy:

In [8]:
len(model.get_weights())
initial_weights = model.get_weights()

In [9]:
threshold = 0.1

In [10]:
# pruning mask

pruning_mask = []


for weights in model.get_weights() :
    mask = abs(weights) > threshold
    pruning_mask.append(mask.astype(np.float32))



In [11]:
pruned_weights = [w*mask for w,mask in zip(initial_weights,pruning_mask)]

model.set_weights(pruned_weights)

In [13]:
loss = tf.keras.losses.MeanSquaredError()
opti = tf.keras.optimizers.Adam(learning_rate=0.001)


In [14]:
for weight, mask in zip(model.trainable_weights, pruning_mask):
    weight = weight*mask

with tf.GradientTape() as tape:
    # Forward pass
    
    outputs = model(x_train)
    # print(outputs)
    loss_value = loss(outputs,y_train)
    

    # Backward pass and weight update
gradients = tape.gradient(loss_value, model.trainable_variables)
opti.apply_gradients(zip(gradients, model.trainable_variables))
# gradients


In [15]:
num_epochs = 10

for epoch in range(num_epochs):
    with tf.GradientTape() as tape:
        # Forward pass
        outputs = model(x_train)
        loss_value = loss(outputs, y_train)
    

    # Backward pass and weight update
    gradients = tape.gradient(loss_value, model.trainable_weights)
    opti.apply_gradients(zip(gradients, model.trainable_weights))

    # freezing the weights 

    for weight, mask in zip(model.trainable_weights, pruning_mask):
        weight.assign(tf.where(mask > 0, weight, 0.0))




In [16]:
model.get_weights()

[array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        , -0.14235725],
        [ 0.        ,  0.        ,  0.11927899, ...,  0.        ,
          0.        , -0.3542399 ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         -0.12122962,  0.19700977],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.39048272],
        [-0.19450563,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]], dtype=float32),
 array([ 0.        ,  0.10396938,  0.13139686,  0.        ,  0.26511854,
         0.        ,  0.        ,  0.28412008,  0.        ,  0.13511324,
         0.        ,  0.        ,  0.2808122 ,  0.12023984,  0.17453533,
         0.13877618,  0.15083528,  0.        , -0.3034049 , -0.56829417,
         0.        ,  0.        ,  0.14321081, -0.36288327,  0.   

In [17]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.0165 - accuracy: 0.8852


[0.01654585637152195, 0.885200023651123]